<a href="https://colab.research.google.com/github/TanJiaTing/AIP/blob/master/Hay_stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-gre5bnbk
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-gre5bnbk
     |████████████████████████████████| 194kB 3.6MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 81kB 5.4MB/s 
     |████████████████████████████████| 2.9MB 11.4MB/s 
     |████████████████████████████████| 194kB 38.4MB/s 
     |████████████████████████████████| 522kB 31.6MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 983kB 40.4MB/s 
     |████████████████████████████████| 5.5MB 20.1MB/s 
     |████████████████████████████████| 778kB 39.7MB/s 
     |████████████████████████████████| 327kB 31.3MB/s 
     |████████████████████████████████| 2.5MB 39.5MB/s 
     |████████████████████████████████| 47.7MB 66kB/s 
     |████████████████████████████████| 

In [2]:

from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers


In [3]:

# In-Memory Document Store
#from haystack.database.memory import InMemoryDocumentStore
#document_store = InMemoryDocumentStore()
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [38]:
# Connect to Elasticsearch
from haystack.database.elasticsearch import ElasticsearchDocumentStore

# We need to set `embedding_field` and `embedding_dim`, when we plan to use a dense retriever
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document", 
                                            embedding_field="embedding", embedding_dim=768)

07/26/2020 09:15:02 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:400 request:0.021s]


In [ ]:

# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>, "text": "<the-actual-text>"}
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Finder)

# Let's have a look at the first 3 entries:
print(dicts[:15])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

07/24/2020 11:21:10 - INFO - haystack.indexing.utils -   Found data stored in `data/article_txt_got`. Delete this first if you really want to fetch new data.


[{'text': '#REDIRECT List of A Song of Ice and Fire characters#R', 'meta': {'name': '486_Ronnet_Connington.txt'}}, {'text': '#REDIRECT List of A Song of Ice and Fire characters#Brynden Rivers', 'meta': {'name': '414_Leaf__character_.txt'}}, {'text': '#REDIRECT World of A Song of Ice and Fire#The Dothraki Sea', 'meta': {'name': '117_Dothraki.txt'}}, {'text': '#REDIRECT List of A Song of Ice and Fire characters#Edmure Tully', 'meta': {'name': '412_Roslin_Tully.txt'}}, {'text': '#REDIRECT World of A Song of Ice and Fire#Oldtown', 'meta': {'name': '113_Oldtown__A_Song_of_Ice_and_Fire_.txt'}}, {'text': '"\'\'\'Oathkeeper\'\'\'" is the fourth episode of the fourth season of HBO\'s fantasy television series \'\'Game of Thrones\'\', and the 34th overall.\nThe episode was written by Bryan Cogman, and directed by Michelle MacLaren. It aired on April 27, 2014. The title refers to the new sword gifted to Brienne of Tarth by Jaime Lannister and the themes of duty that propel the episode. The episod

07/24/2020 11:21:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.187s]
07/24/2020 11:21:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.152s]
07/24/2020 11:21:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.222s]
07/24/2020 11:21:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.204s]
07/24/2020 11:21:12 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.165s]
07/24/2020 11:21:12 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.085s]


In [5]:
from google.colab import drive
drive.mount('/drive')
#temp = convert_files_to_dicts(dir_path="data/custom", clean_func=clean_wiki_text, split_paragraphs=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [12]:
dicts = convert_files_to_dicts(dir_path='data', clean_func=clean_wiki_text, split_paragraphs=True)
dicts

[{'meta': {'name': 'summary.txt'},
  'text': "4.1.2 PART 2 — EN-ROUTE (ENR) Part 2 consists of seven sections containing information briefly described hereafter\nGEN 1.1 DESIGNATED AUTHORITIES The authority responsible for civil aviation in Singapore is the Civil Aviation Authority of Singapore under the Ministry of Transport\n1.2 Aircraft flying into or departing from Singapore territory shall make their first landing at, or final departure from an international aerodrome (see AIP Singapore page AD 1.3-1 and section AD 2)\n1.3 Notwithstanding the regulations relating to civil aviation over Singapore territory, aircraft operators should consult the respective AIPs for other documentary and / or permit requirements for flights intending to enter, depart, and / or overfly the sovereign airspaces of States along the planned flight routes\n2 APPLICATION FOR SLOTS AT SINGAPORE CHANGI AIRPORT 2.1 Singapore Changi Airport is a slot coordinated airport, with Changi Airport Group (CAG) as the S

In [36]:
file1 = open("/drive/My Drive/data/summary.txt","r+") 
string = file1.read()
paragraphs =string.split('.\n')
list_par=[]
for p in range(len(paragraphs)):
  #Remove headers and footers
  #if ('©' in paragraphs[p]) or ('AIP SingaporeGEN' in paragraphs[p]):
    #continue
  #turn lists into sentences by replacing - with ,
  paragraphs[p] = paragraphs[p].replace(';\n -',',')
  #first element of list does not require a comma
  paragraphs[p]= paragraphs[p].replace('\n -','')
  paragraphs[p]=paragraphs[p].replace(';','')
  result = paragraphs[p].split('\n')
  for r in range(len(result)):
    #Remove whitespace between lines
    result[r] = result[r].strip()
    #Remove whitespace between words
    words = result[r].split(' ')
    s = ' '
    result[r] = s.join([word for word in words if word != ''])
  paragraphs[p] = s.join(result)
  paragraphs[p]=paragraphs[p].replace('\n','')
  paragraphs[p] = paragraphs[p].replace('°',' degrees ')
  list_par.append(paragraphs[p].strip())
sep = '\n '
passage = sep.join(list_par)
dict1= [{'meta':{'name':'summary.txt'},'text':passage}]
dict1

[{'meta': {'name': 'summary.txt'},
  'text': "1 INTRODUCTION 1.1 International flights into, from or over Singapore territory shall be subject to the current Singapore regulations relating to civil aviation. These regulations correspond in all essentials to the Standards and Recommended Practices contained in Annex 9 to the Convention on International Civil Aviation\n 1.2 Aircraft flying into or departing from Singapore territory shall make their first landing at, or final departure from an international aerodrome (see AIP Singapore page AD 1.3-1 and section AD 2)\n 1.3 Notwithstanding the regulations relating to civil aviation over Singapore territory, aircraft operators should consult the respective AIPs for other documentary and / or permit requirements for flights intending to enter, depart, and / or overfly the sovereign airspaces of States along the planned flight routes\n 2 APPLICATION FOR SLOTS AT SINGAPORE CHANGI AIRPORT 2.1 Singapore Changi Airport is a slot coordinated airpo

In [39]:
document_store.write_documents(dict1)
#paragraphs
#dict1

07/26/2020 09:15:12 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.068s]


In [40]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store, embedding_model="dpr-bert-base-nq",
                                  do_lower_case=True, use_gpu=True)

# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

07/26/2020 09:15:27 - INFO - haystack.retriever.dpr_utils -   Loading saved model from models/dpr/checkpoint/retriever/single/nq/bert-base-encoder.cp
07/26/2020 09:15:51 - INFO - haystack.retriever.dense -   Loaded encoder params:  {'do_lower_case': True, 'pretrained_model_cfg': 'bert-base-uncased', 'encoder_model_type': 'hf_bert', 'pretrained_file': None, 'projection_dim': 0, 'sequence_length': 256}
07/26/2020 09:16:11 - INFO - haystack.retriever.dense -   Loading saved model state ...
07/26/2020 09:16:12 - INFO - haystack.retriever.dense -   Loading saved model state ...
07/26/2020 09:16:13 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=5m&size=1000 [status:200 request:0.983s]
07/26/2020 09:16:13 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.008s]
07/26/2020 09:16:13 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.002s]
07/26/2020 09:16:13 - INFO - haystack.databas

In [9]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/bert-large-uncased-whole-word-masking-squad2", use_gpu=True)

07/26/2020 08:44:09 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
07/26/2020 08:44:09 - INFO - farm.infer -   Could not find `deepset/bert-large-uncased-whole-word-masking-squad2` locally. Try to download from model hub ...
07/26/2020 08:44:10 - INFO - filelock -   Lock 139668043148872 acquired on /root/.cache/torch/transformers/096f66c5a6e2e4fb4ed910fbd3a41a8206f44edb0c9733d3eb62269cd45ad38f.2bcd58a3e601d95c0636a5bc716b70e64e819abb894ef808d4fc209572800dc2.lock


07/26/2020 08:44:10 - INFO - filelock -   Lock 139668043148872 released on /root/.cache/torch/transformers/096f66c5a6e2e4fb4ed910fbd3a41a8206f44edb0c9733d3eb62269cd45ad38f.2bcd58a3e601d95c0636a5bc716b70e64e819abb894ef808d4fc209572800dc2.lock


07/26/2020 08:44:10 - INFO - filelock -   Lock 139667751629992 acquired on /root/.cache/torch/transformers/7fd93eb96e5abd383bbdc176f634bd8c724fbac421d6b365c3669bb53d5f4879.742352a3b720b1222c541027ae88b416e5eb7d9f4323b38a6d7368b5598b14ef.lock


07/26/2020 08:44:45 - INFO - filelock -   Lock 139667751629992 released on /root/.cache/torch/transformers/7fd93eb96e5abd383bbdc176f634bd8c724fbac421d6b365c3669bb53d5f4879.742352a3b720b1222c541027ae88b416e5eb7d9f4323b38a6d7368b5598b14ef.lock


07/26/2020 08:44:53 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
07/26/2020 08:45:17 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
07/26/2020 08:45:20 - INFO - filelock -   Lock 139667749749536 acquired on /root/.cache/torch/transformers/1a2ae6d1bcb99ae1ff81986f23c8ce7735cd271a2b5cf46eaa461439f648b0e2.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


07/26/2020 08:45:21 - INFO - filelock -   Lock 139667749749536 released on /root/.cache/torch/transformers/1a2ae6d1bcb99ae1ff81986f23c8ce7735cd271a2b5cf46eaa461439f648b0e2.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
07/26/2020 08:45:21 - INFO - filelock -   Lock 139668041627240 acquired on /root/.cache/torch/transformers/76839de60d3432118c9701e0c084d5ef20d84044bfc08d04f3a0f1ecd4fe59ec.3889713104075cfee9e96090bcdd0dc753733b3db9da20d1dd8b2cd1030536a2.lock


07/26/2020 08:45:21 - INFO - filelock -   Lock 139668041627240 released on /root/.cache/torch/transformers/76839de60d3432118c9701e0c084d5ef20d84044bfc08d04f3a0f1ecd4fe59ec.3889713104075cfee9e96090bcdd0dc753733b3db9da20d1dd8b2cd1030536a2.lock
07/26/2020 08:45:21 - INFO - filelock -   Lock 139668041627856 acquired on /root/.cache/torch/transformers/ec852705de244031a7f6052b4567bb1db19a525413d70ffc421854df218b32ca.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock


07/26/2020 08:45:21 - INFO - filelock -   Lock 139668041627856 released on /root/.cache/torch/transformers/ec852705de244031a7f6052b4567bb1db19a525413d70ffc421854df218b32ca.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock
07/26/2020 08:45:22 - INFO - filelock -   Lock 139668041627912 acquired on /root/.cache/torch/transformers/d321263458c964122ba0d7c0294bdd2f40d6f812aaaded8b893fa3c0c1068ae7.1ade4e0ac224a06d83f2cb9821a6656b6b59974d6552e8c728f2657e4ba445d9.lock


07/26/2020 08:45:22 - INFO - filelock -   Lock 139668041627912 released on /root/.cache/torch/transformers/d321263458c964122ba0d7c0294bdd2f40d6f812aaaded8b893fa3c0c1068ae7.1ade4e0ac224a06d83f2cb9821a6656b6b59974d6552e8c728f2657e4ba445d9.lock


07/26/2020 08:45:22 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
07/26/2020 08:45:22 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
07/26/2020 08:45:22 - INFO - farm.infer -    0 
07/26/2020 08:45:22 - INFO - farm.infer -   /w\
07/26/2020 08:45:22 - INFO - farm.infer -   /'\
07/26/2020 08:45:22 - INFO - farm.infer -   


In [41]:
finder = Finder(reader, retriever)

In [32]:
#individual qn
qn = "What details of flight operations do operators need to include in their submission?"
prediction = finder.get_answers(question=qn, top_k_retriever=10, top_k_reader=1)
print('Question: ' + qn)
print('Predicted: ' + prediction['answers'][0]['answer'])
print('context: ' + prediction['answers'][0]['context'])

/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
07/26/2020 09:09:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.009s]
07/26/2020 09:09:15 - INFO - haystack.finder -   Reader is looking for detailed answer in 44584 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.29s/ Batches]

Question: What details of flight operations do operators need to include in their submission?
Predicted: DOCUMENTARY REQUIREMENTS FOR CLEARANCE OF AIRCRAFT
context:  having the business management of that aircraft
 DOCUMENTARY REQUIREMENTS FOR CLEARANCE OF AIRCRAFT Same requirements as for SCHEDULED FLIGHTS
 5.3 P


In [42]:

import pandas as pd
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
#prediction = finder.get_answers(question="Which documents need to be lodged with CAAS?", top_k_retriever=50, top_k_reader=10)

#read queries
queries = pd.read_csv('/drive/My Drive/data/qns.csv') 
queries['PageNum'] = pd.Series(queries['PageNum']).fillna(method='ffill')
queries['Context'] = pd.Series(queries['Context']).fillna(method='ffill')
for index in range(len(queries['Question'])):
  qn = queries['Question'][index]
  prediction = finder.get_answers(question=qn, top_k_retriever=30, top_k_reader=1)
  print('Question: ' + qn)
  print('Predicted: ' + prediction['answers'][0]['answer'])
  print('Actual: ' + queries['Answer'][index])
  print('context: ' + prediction['answers'][0]['context'])



/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
07/26/2020 09:16:32 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.009s]
07/26/2020 09:16:32 - INFO - haystack.finder -   Reader is looking for detailed answer in 88992 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.16s/ Batches]
07/26/2020 09:16:46 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.020s]
07/26/2020 09:16:46 - INFO - haystack.finder -   Reader is looking for detailed answer in 88992 chars ...


Question: When is the earliest date for operators of non-commercial flights to submit their slot requests?
Predicted: 7 calendar days
Actual: no earlier than 7 calendar days prior to operation
context:  their slot requests to the Changi Slot Coordinator no earlier than 7 calendar days and but no later than 24 hours prior to the operation of the fligh


Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.19s/ Batches]
07/26/2020 09:17:00 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.008s]
07/26/2020 09:17:00 - INFO - haystack.finder -   Reader is looking for detailed answer in 88992 chars ...


Question: Who should operators or agents of non-commercial flights submit their slot requests to?
Predicted: the Changi Slot Coordinator
Actual: The Changi Slot Coordinator
context: nd non-commercial flights shall submit their slot requests to the Changi Slot Coordinator no earlier than 7 calendar days and but no later than 24 hou


Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.21s/ Batches]
07/26/2020 09:17:14 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.009s]
07/26/2020 09:17:14 - INFO - haystack.finder -   Reader is looking for detailed answer in 88992 chars ...


...

Question: Which aircrafts does paragraph 50D of the Air Navigation Order apply to?
Predicted: all aircraft flying to, from or over the Republic of Singapore
Actual: all aircraft flying to, from or over the Republic of Singapore, and to Singapore registered aircraft wherever they may be
context: h may be imposed. This provision applies to all aircraft flying to, from or over the Republic of Singapore, and to Singapore registered aircraft where


Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.16s/ Batches]
07/26/2020 09:23:04 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.007s]
07/26/2020 09:23:04 - INFO - haystack.finder -   Reader is looking for detailed answer in 88992 chars ...


Question: What documents are required to carry dangerous goods in an aircraft?
Predicted: a dangerous goods permit
Actual: dangerous goods permit granted by the Director-General of Civil Aviation
context: raft unless the operator of the aircraft has been granted with a dangerous goods permit granted by the Director-General of Civil Aviation and in accor


Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.21s/ Batches]

Question: Where should operators who wish to carry dangerous goods submit their applications to?
Predicted: Arms and Explosives Branch, Commissioner of Singapore Police
Actual: Flight Standards Division; Civil Aviation Authority of Singapore Singapore Changi Airport; P.O.Box 1, Singapore 918141; FAX: (65) 65456519; TEL: (65) 65413487
context: ained and applications should be made to the Arms and Explosives Branch, Commissioner of Singapore Police, Block J, Kinloss Complex, No 3 Ladyhill Roa


In [43]:
#Evaluation
from collections import Counter
import string
import re

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [56]:
scores = {}
for index in range(len(queries['Question'])):
  qn = queries['Question'][index]
  prediction = finder.get_answers(question=qn, top_k_retriever=30, top_k_reader=1)
  task = queries['TaskType'][index]
  if task not in scores:
    scores[task] = []
  predicted = prediction['answers'][0]['answer']
  actual = queries['Answer'][index]
  scores[task].append(f1_score(predicted,actual))
print(scores)

/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
07/26/2020 09:47:46 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.015s]
07/26/2020 09:47:46 - INFO - haystack.finder -   Reader is looking for detailed answer in 88992 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.20s/ Batches]
07/26/2020 09:48:00 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.012s]
07/26/2020 09:48:00 - INFO - haystack.finder -   Reader is looking for detailed answer in 88992 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:06<00:00,  3.

{1: [0.5, 1.0, 1.0, 1.0, 1.0, 0, 0.4545454545454546, 0, 0.5454545454545454, 0.35294117647058826, 0.6666666666666666, 1.0, 0.3, 1.0, 0.36363636363636365, 0.14285714285714288, 0.34285714285714286, 0.6896551724137931, 0.5, 0.13793103448275862], 9: [0, 0.625, 0], 8: [0.03571428571428571, 0.15384615384615385, 1.0, 1.0, 1.0, 0]}


In [78]:
# from google.colab import files
# sep = ' '
# text = sep.join(list_par)

# with open('example.txt', 'w') as f:
#   f.write(text)

# files.download('example.txt')
#document_store.write_documents(string)
# b = '3'
# a = {}
# if b not in a:
#   a[b] = []
# a[b].append(4)
# a
from __future__ import print_function
import numpy as np
total_len = total_score = 0
for k in scores.keys():
  total_len += len(scores[k])
  total_score += sum(scores[k])
  print(k)
  print(np.mean(scores[k]))
  s = "F1 score for Task {0} is {1}".format(k, np.mean(scores[k]))
  print(s)
print("Overall F1 score : " + str(total_score/total_len))

1
0.5498272349692228
F1 score for Task 1 is 0.5498272349692228
9
0.20833333333333334
F1 score for Task 9 is 0.20833333333333334
8
0.5315934065934066
F1 score for Task 8 is 0.5315934065934066
Overall F1 score : 0.510727763411893
